In [1]:
import numpy
import pandas

import conntility

from scipy.spatial.distance import cdist
import tqdm

from matplotlib import pyplot as plt

## Part 1:
Measure the strengths of statistical interactions in connectivity based on individual morphologies that are not capture by a fourth-order simplifed model

In [23]:
fn_rat = "/gpfs/bbp.cscs.ch/project/proj159/home/barros/conn_matrix/Rat_623um_squared_struc_conmat_filtered_compressed.h5"
fn_human = "/gpfs/bbp.cscs.ch/project/proj159/home/barros/conn_matrix/Human_960um_squared_struc_conmat_filtered_compressed.h5"

loaded = "human"  # Set to human to analyze that instead
extent_xz = 900
nbins_xz = 31
extent_y = 700
nbins_y = 41

if loaded == "rat":
    M_h = conntility.ConnectivityMatrix.from_h5(fn_rat)
    col_y = "depth"
    col_xz = ["ss_flat_x", "ss_flat_y"]
elif loaded == "human":
    M_h = conntility.ConnectivityMatrix.from_h5(fn_human)
    col_y = "y"
    col_xz = ["x", "z"]

dbins_xz = numpy.hstack([[0, 1E-12], numpy.linspace(1E-12, extent_xz, nbins_xz)])
binid_xz = numpy.arange(0, nbins_xz + 1)

dbins_y = numpy.linspace(-extent_y, extent_y, nbins_y)
binid_y = numpy.arange(0, nbins_y + 1)

M_h.vertices["mtype"].value_counts()

mtype
L23_PTPC    14994
L23_STPC     5028
IN              0
L4_PC           0
L56_PC          0
Name: count, dtype: int64

In [ ]:
from scipy.spatial import KDTree

_coords = col_xz + [col_y]
tree = KDTree(M_h.vertices[_coords].values)

_, nn_id = tree.query(M_h.vertices[_coords], k=2)
nn_id = nn_id[:, 1]  # nn_id[:, 0] is the original node, which has distance 0. nn_id[:, 1] is neighbor

# Lookup from pre / post ids to edge ids
edge_id_lookup = M_h._edge_indices.reset_index(drop=True).reset_index().set_index(["row", "col"])["index"]

In [ ]:
def for_pre_chunk(chunk_pre):
    # Which offset bin the pairs fall into
    Dxz = cdist(M_h.vertices.iloc[chunk_pre][col_xz], M_h.vertices[col_xz]) # PRE X POST
    Dxz = numpy.digitize(Dxz, dbins_xz) - 2  # -2 means distance = 0 will be bin id -1

    Dy = M_h.vertices.iloc[chunk_pre][[col_y]].values - M_h.vertices[[col_y]].values.transpose() # PRE X POST
    Dy = numpy.digitize(Dy, dbins_y) - 1  # NOTE: Negative values -> upwards connection
    
    # Numer of touches between them
    j, i = numpy.meshgrid(numpy.arange(len(M_h)), chunk_pre)
    assert j.shape == Dy.shape
    con_index = pandas.MultiIndex.from_frame(pandas.DataFrame({"row": i.flatten(), "col": j.flatten()}))
    edge_ids = edge_id_lookup.reindex(con_index, fill_value=-1).values
    
    touch_count = numpy.zeros(len(edge_ids))
    v = edge_ids > 0
    touch_count[v] = M_h.edges["count"].values[edge_ids[v]]
    
    # Number of touches with nearest neighbor (pre)
    _pre = nn_id[i.flatten()]; _post = j.flatten()
    con_index = pandas.MultiIndex.from_frame(pandas.DataFrame({"row": _pre, "col": _post}))
    edge_ids = edge_id_lookup.reindex(con_index, fill_value=-1).values
    
    touch_count_nnpre = numpy.zeros(len(edge_ids))
    v = edge_ids > 0
    touch_count_nnpre[v] = M_h.edges["count"].values[edge_ids[v]]
    collision_pre = _pre != _post
    
    # Number of touches with nearest neighbor (post)
    _pre = i.flatten(); _post = nn_id[j.flatten()]
    con_index = pandas.MultiIndex.from_frame(pandas.DataFrame({"row": _pre, "col": _post}))
    edge_ids = edge_id_lookup.reindex(con_index, fill_value=-1).values
    
    touch_count_nnpost = numpy.zeros(len(edge_ids))
    v = edge_ids > 0
    touch_count_nnpost[v] = M_h.edges["count"].values[edge_ids[v]]
    collision_post = _pre != _post
    
    # Count instances of each
    ret_pre = pandas.DataFrame({
        "xz": Dxz.flatten()[collision_pre],
        "y": Dy.flatten()[collision_pre],
        "touches_pair": touch_count[collision_pre],
        "touches_nn_pre": touch_count_nnpre[collision_pre],
    }).value_counts()
    
    ret_post = pandas.DataFrame({
        "xz": Dxz.flatten()[collision_post],
        "y": Dy.flatten()[collision_post],
        "touches_pair": touch_count[collision_post],
        "touches_nn_post": touch_count_nnpost[collision_post],
    }).value_counts()
    return ret_pre, ret_post
    


In [ ]:
#In each offset-bin: How many pairs exist?
full_master_pre = []
full_master_post = []
chunk_sz = 1000
chunking = numpy.arange(0, len(M_h) + chunk_sz, chunk_sz)

chunk_pre = numpy.arange(chunking[0], numpy.minimum(chunking[1], len(M_h)))
full_master_pre, full_master_post = for_pre_chunk(chunk_pre)

for a, b in tqdm.tqdm(list(zip(chunking[1:-1], chunking[2:]))):
    chunk_pre = numpy.arange(a, numpy.minimum(b, len(M_h)))
    new_master_pre, new_master_post = for_pre_chunk(chunk_pre)
    full_master_pre = full_master_pre.add(new_master_pre, fill_value=0)
    full_master_post = full_master_post.add(new_master_post, fill_value=0)


In [ ]:
full_master_pre = full_master_pre.reset_index()
full_master_post = full_master_post.reset_index()

bin_centers_y = numpy.hstack([
    0.5 * (dbins_y[:-1] + dbins_y[1:]),
    dbins_y[-1] + 0.5 * numpy.mean(numpy.diff(dbins_y))
])
bin_centers_xz = numpy.hstack([
    0.5 * (dbins_xz[1:-1] + dbins_xz[2:]),
    dbins_xz[-1] + 0.5 * numpy.mean(numpy.diff(dbins_xz[1:]))
])

In [ ]:
_v = full_master_pre["xz"] >= 0
full_master_pre["xz"][_v] = bin_centers_xz[full_master_pre["xz"][_v]]
full_master_pre["y"] = bin_centers_y[full_master_pre["y"]]

_v = full_master_post["xz"] >= 0
full_master_post["xz"][_v] = bin_centers_xz[full_master_post["xz"][_v]]
full_master_post["y"] = bin_centers_y[full_master_post["y"]]

In [ ]:
out_fn = "/gpfs/bbp.cscs.ch/project/proj159/home/reimann/connectivity_higher_order_effect.h5"

full_master_pre.to_hdf(out_fn, key="{0}/pre".format(loaded))
full_master_post.to_hdf(out_fn, key="{0}/post".format(loaded))

## Part 2:
The above characterizes statistical interactions within one spatial bin. Here, we consider interactions _between_ bins. 

In [24]:
# NOTE: I repeat some of the parameterization from Part 1. In case you want to run only this part.
extent_xz = 900
nbins_xz = 31
extent_y = 700
nbins_y = 41

dbins_xz = numpy.hstack([[0, 1E-12], numpy.linspace(1E-12, extent_xz, nbins_xz)])
binid_xz = numpy.arange(0, nbins_xz + 1)

dbins_y = numpy.linspace(-extent_y, extent_y, nbins_y)
binid_y = numpy.arange(0, nbins_y + 1)

edge_id_lookup = M_h._edge_indices.reset_index(drop=True).reset_index().set_index(["row", "col"])["index"]

def for_chunk(chunk, interaction_for="pre"):
    # Which offset bin the pairs fall into
    if interaction_for == "pre":
        Dxz = cdist(M_h.vertices.iloc[chunk][col_xz], M_h.vertices[col_xz]) # PRE X POST
        Dxz = numpy.digitize(Dxz, dbins_xz) - 2  # -2 means distance = 0 will be bin id -1

        Dy = M_h.vertices.iloc[chunk][[col_y]].values - M_h.vertices[[col_y]].values.transpose() # PRE X POST
        Dy = numpy.digitize(Dy, dbins_y) - 1  # NOTE: Negative values -> upwards connection
        
        j, i = numpy.meshgrid(numpy.arange(len(M_h)), chunk)
        node_id = i.flatten()
    elif interaction_for == "post":
        Dxz = cdist(M_h.vertices[col_xz], M_h.vertices.iloc[chunk][col_xz]) # PRE X POST
        Dxz = numpy.digitize(Dxz, dbins_xz) - 2  # -2 means distance = 0 will be bin id -1

        Dy = M_h.vertices[[col_y]].values - M_h.vertices.iloc[chunk][[col_y]].values.transpose() # PRE X POST
        Dy = numpy.digitize(Dy, dbins_y) - 1  # NOTE: Negative values -> upwards connection
        
        j, i = numpy.meshgrid(chunk, numpy.arange(len(M_h)))
        node_id = j.flatten()
    
    # Numer of touches between them
    assert j.shape == Dy.shape
    con_index = pandas.MultiIndex.from_frame(pandas.DataFrame({"row": i.flatten(), "col": j.flatten()}))
    edge_ids = edge_id_lookup.reindex(con_index, fill_value=-1).values
    
    touch_count = numpy.zeros(len(edge_ids))
    v = edge_ids > 0
    touch_count[v] = M_h.edges["count"].values[edge_ids[v]]
    touch_count.reshape(Dy.shape)
        
    # Count instances of each
    ret = pandas.DataFrame({
        "xz": Dxz.flatten(),
        "y": Dy.flatten(),
        "touches_pair": touch_count,
        "node_id": node_id,
    }).value_counts()
    
    return ret
    


In [25]:
df_for_pre = []
df_for_post = []

chunk_sz = 2000
chunking = numpy.arange(0, len(M_h) + chunk_sz, chunk_sz)

for a, b in tqdm.tqdm(list(zip(chunking[:-1], chunking[1:]))):
    chunk = numpy.arange(a, numpy.minimum(b, len(M_h)))
    df_for_pre.append(for_chunk(chunk, interaction_for="pre"))
    df_for_post.append(for_chunk(chunk, interaction_for="post"))
    
df_for_pre = pandas.concat(df_for_pre, axis=0)
df_for_post = pandas.concat(df_for_post, axis=0)

100%|██████████| 11/11 [05:54<00:00, 32.22s/it]


In [30]:
# We introduce a threshold for minimum number of touches to make the results sparser. That enables simplex counting later
thresh = 5  # HOW MANY TOUCHES NEED TO BE REACHED FOR A CONNECTION

# For each pre- or post-neuron: How many partners does it have in each offset bin (N) and what fraction is connected (P)?
def count_n_and_p(df_in):
    M = df_in.pivot(index="touches_pair", columns="node_id", values="count") # "node_id"
    V = pandas.concat([M.sum(axis=0), M.loc[thresh:].sum(axis=0) / M.sum(axis=0)], axis=1, keys=["N", "P"])
    return V

df_n_p_pre = df_for_pre.reset_index().groupby(["xz", "y"]).apply(count_n_and_p)
df_n_p_post = df_for_post.reset_index().groupby(["xz", "y"]).apply(count_n_and_p)

In [31]:
df_n_p_post.head()

N    P
xz y  node_id          
-1 20 0        1.0  0.0
      1        1.0  0.0
      2        1.0  0.0
      3        1.0  0.0
      4        1.0  0.0

In [32]:
# Overall mean connection probability in each offset bin
mn_P_pre = df_n_p_pre.drop(-1).groupby(["xz", "y"]).apply(lambda _x: (_x["N"] * _x["P"]).sum() / _x["N"].sum())
mn_P_post = df_n_p_post.drop(-1).groupby(["xz", "y"]).apply(lambda _x: (_x["N"] * _x["P"]).sum() / _x["N"].sum())

# What is the correlation between observed values of "P" over pre- or post-neurons in a given offset bin?
def observations_to_cc_of_offset_bins(df_in, mode="valid"):
    df_cc = df_in.reset_index()
    # The -1 bin is the pair of one neuron with itself. Exclude.
    df_cc = df_cc.loc[df_cc["xz"] >= 0]
    # columns: offset bin. index: neuron. values: observed "P", or NaN if the neuron has no other neuron at that offset
    df_cc = df_cc.pivot(columns=["xz", "y"], index="node_id", values="P")
    # How to deal with NaN values? The default behavior of pandas is to ignore them, i.e. calculate correlation only
    # for observations where neither of a pair is NaN. 
    # A more conservative approach is to assume that the neuron would have the overall mean connection probability.
    # Finally we can add a small, random noise to all observations. That way, bins with all zeros will show up
    # to have zero correlation with all others. Else they will have NaN correlation
    if mode == "valid":
        return df_cc.corr()
    else:
        return (df_cc.fillna(mn_P) + 1E-9 * numpy.random.rand(*df_cc.shape)).corr()
    
CC_pre = observations_to_cc_of_offset_bins(df_n_p_pre)
CC_post = observations_to_cc_of_offset_bins(df_n_p_post)

In [33]:
bin_centers_y = numpy.hstack([
    0.5 * (dbins_y[:-1] + dbins_y[1:]),
    dbins_y[-1] + 0.5 * numpy.mean(numpy.diff(dbins_y))
])
bin_centers_xz = numpy.hstack([
    0.5 * (dbins_xz[1:-1] + dbins_xz[2:]),
    dbins_xz[-1] + 0.5 * numpy.mean(numpy.diff(dbins_xz[1:]))
])

CC_post = CC_post.drop(-1, axis=0, level=1).drop(-1, axis=1, level=1)
CC_pre = CC_pre.drop(-1, axis=0, level=1).drop(-1, axis=1, level=1)

In [34]:
def translate_index(idx):
    idx = idx.to_frame().reset_index(drop=True)
    idx["xz"] = bin_centers_xz[idx["xz"]]
    idx["y"] = bin_centers_y[idx["y"]]
    return pandas.MultiIndex.from_frame(idx)

CC_post.index = translate_index(CC_post.index)
CC_post.columns = translate_index(CC_post.columns)
CC_pre.index = translate_index(CC_pre.index)
CC_pre.columns = translate_index(CC_pre.columns)

In [35]:
out_fn = "/gpfs/bbp.cscs.ch/project/proj159/home/reimann/connectivity_higher_order_interactions.h5"

CC_pre.to_hdf(out_fn, key="{0}/{1}/pre".format(loaded, thresh))
CC_post.to_hdf(out_fn, key="{0}/{1}/post".format(loaded, thresh))

In [55]:
mn_edge_count_pre = df_n_p_pre.drop(-1, axis=0, level=0, errors="ignore").drop(-1, axis=0, level=1, errors="ignore").groupby(["xz", "y"]).apply(lambda _x: _x["N"].mean())
mn_edge_count_pre.index = translate_index(mn_edge_count_pre.index)
mn_edge_count_post = df_n_p_post.drop(-1, axis=0, level=0, errors="ignore").drop(-1, axis=0, level=1, errors="ignore").groupby(["xz", "y"]).apply(lambda _x: _x["N"].mean())
mn_edge_count_post.index = translate_index(mn_edge_count_post.index)

In [60]:
out_fn = "/gpfs/bbp.cscs.ch/project/proj159/home/reimann/connectivity_higher_order_interactions.h5"

mn_edge_count_pre.to_hdf(out_fn, key="{0}/{1}/pre_N".format(loaded, thresh))
mn_edge_count_post.to_hdf(out_fn, key="{0}/{1}/post_N".format(loaded, thresh))